# Lab 4

I am going to design a database for metabolism by first creating three relational tables for genes, enzymes, and pathways.  I will also create three associative tables to link the information across tables.

Below are the three relational tables with appropriate fields for each row.

Q.Some enzymes belong to the same pathway.  Some might belong to multiple pathways. Is it one to many or many to many?  
A. According to my findings it is many-to-many.
Q.There is an order to enzymes within the same pathway.  How can that order be represented in table?  
A. We can add a separate column that indicates the enzyme order number within the same pathway.  Columns: pathway id, enzyme id, enzyme order number.
Q.Genes in the gene table encode enzymes.  How can this be represented? Is this a one-to-one, one-to-many, or many-to-many and in which direction? 
A. It is one-to-many in the direction of enzymes to genes since multiple genes encode the same enzyme.  

In [146]:
import sqlite3
conn = sqlite3.connect("New.db")
c = conn.cursor()

# c.execute("""DROP TABLE genelist""")
# c.execute("""DROP TABLE pathwaysDB""")
# c.execute("""DROP TABLE enzymeDB""")
# c.execute("""DROP TABLE pathways_to_enzymes""")
# c.execute("""DROP TABLE pathway_enzyme_order""")
# c.execute("""DROP TABLE genes_to_enzymes""")

##################### Relational Tables #####################
c.execute("""CREATE TABLE genes (id INT, 
                                name TEXT,
                                description TEXT,
                                organism TEXT,
                                nucleotide_sequence VARCHAR(1));""")

c.execute("""CREATE TABLE pathways (id INT, 
                                name TEXT,
                                description TEXT);""")

c.execute("""CREATE TABLE enzyme (id INT, 
                                name TEXT,
                                EC number TEXT);""")

##################### Associative Tables #####################

c.execute("""CREATE TABLE pathways_to_enzymes (pathway_id INT, 
                                enzyme_id INT);""")

c.execute("""CREATE TABLE pathway_enzyme_order (pathway_id INT, 
                                order_no INT,
                                enzyme_id INT);""")

c.execute("""CREATE TABLE genes_to_enzymes (gene_id INT, 
                                enzyme_id INT);""")
conn.commit()

OperationalError: table genes already exists

Note: I ran into some trouble using SQL because at first my database kept locking and then I was not able to add rows.  Due to limited time, I show what I would have done make my databases.

I used the KEGG site to choose four enzymes from each of the three pathways specified in Lab 4.  They were noted in the diagrams by EC number.  I then input these numbers into the NCBI database to get official names.  The number in parantheses shows the order of enzymes within the same pathway as noted in the KEGG diagrams.

Three pathways:

Glycolysis
	-1.2.1.9(3)
	glyceraldehyde-3-phosphate dehydrogenase
	-5.3.1.9(2)
	glucose-6-phosphate isomerase
	-5.4.2.2 (1)
	phosphoglucomutase
	-4.2.1.11(4)
	phosphopyruvate hydratase
    
TCA cycle
	- 1.1.1.37 (4)
	malate dehydrogenase
	- 1.1.1.42(1)
	isocitrate dehydrogenase
	- 2.3.1.61 (2)
	dihydrolipoyllysine-residue succinyltransferase
	- 4.2.1.2 (3)
	Fumarate hydratase

Pentose Phosphate
	-1.2.1.9 (4)
	glyceraldehyde-3-phosphate dehydrogenase
	-5.3.1.9 (1)
	glucose-6-phosphate isomerase
	-2.2.1.1 (2)
	transketolase
	-4.2.1.12 (3)
    phosphogluconate dehydratase

I purposefully chose similar enzymes in different pathways to show the many-to-many relationship between enzymes and pathways.

I used Entrez to search for genbank files of each chosen enzyme in each species (Homo sapiens, E.coli, and Drosophila).

I then compiled all of these files into one large genbank file using SeqIO.write.

In [134]:
from Bio import Entrez
from Bio import SeqIO

Entrez.email = 'karinna_vivanco@berkley.edu'

##################################################################################################################
#Making lists of search terms in each category to input to Entrez
species = ['homo sapiens', 'E.coli', 'Drosophila']
enzymes = ['glyceraldehyde-3-phosphate dehydrogenase',
          'glucose-6-phosphate isomerase',
          'phosphoglucomutase',
          'phosphopyruvate hydratase',
          'malate dehydrogenase',
          'isocitrate dehydrogenase', 
          'dihydrolipoyllysine-residue succinyltransferase',
          'fumarate hydratase',
          'transketolase',
          'phosphogluconate dehydratase']
%store enzymes
EC_numbers = {'glyceraldehyde-3-phosphate dehydrogenase':
              '1.2.1.9',
          'glucose-6-phosphate isomerase':
              '5.3.1.9',
          'phosphoglucomutase':
              '5.4.2.2',
          'phosphopyruvate hydratase':
              '4.2.1.11',
          'malate dehydrogenase':
              '1.1.1.37',
          'isocitrate dehydrogenase':
              '1.1.1.42', 
          'dihydrolipoyllysine-residue succinyltransferase':
              '2.3.1.61',
          'fumarate hydratase':
              '4.2.1.2',
          'transketolase':
              '2.2.1.1',
          'phosphogluconate dehydratase':
             '4.2.1.12'}
%store EC_numbers
#creating a list of sequences from genbank files for compiling into one genbank file
recs= []

#Iterates through all search term combinations(organism and enzyme)
for S in species:
    for enzyme in enzymes:
        #Searches Entrez and returns top 5 results
        handle = Entrez.esearch(db = 'nucleotide', 
                        retmax = 1,
                       term= (S +" [ORGN] "+ enzyme),
                       sort = 'relevance', 
                        idtype= 'acc')
        searchInfo = Entrez.read(handle) #dictionary containing info from the search, including accession numbers
        handle.close()
        #Fetches the GenBank file from the top match in the search
        handle2 = Entrez.efetch(db = 'nucleotide', id = searchInfo['IdList'][0], rettype='gb', retmode = 'text')
        #Uses a sequence iterator to add to list of SeqRecord Objects
        for rec in SeqIO.parse(handle2, "gb"):
            recs.append(rec)   
        handle2.close()

%store recs

#Writes a gb file containing all entries
SeqIO.write(recs,open("MasterGenBank.gb","w"), "gb")


Stored 'enzymes' (list)
Stored 'EC_numbers' (dict)
Stored 'recs' (list)


30

After going through the Master GenBank file, I realized that some of the entries showed whole genomes.  I went through each of these manually and changed the sequences to the correct gene subsequence. I reqrote the genbank file with these updates.

In [136]:
from Bio import SeqIO
import sqlite3
    
recs[2].seq = recs[2].seq[1655936:1657564]
recs[9].seq = recs[9].seq[994321:996165]
recs[11].seq = recs[11].seq[42301:43950]
recs[13].seq = recs[13].seq[90074:91372]
recs[16].seq = recs[16].seq[2253268:2254422]
recs[18].seq = recs[18].seq[391674:393677]
recs[19].seq = recs[19].seq[5049987:5051393]


SeqIO.write(recs,open("MasterGenBankUPDATE.gb","w"), "gb")



30

I ran into trouble below inserting information into my tables.  It worked at times but then when I re-ran the cell later on, it did not work anymore.  I ran out of time to troubleshoot.  Below I show how I would add data to the corresponding tables.  I would add the relationships to the associative tables manually by following my list above, with pathways, enzymes, and enzyme order.  I could also add the gene to enzyme relatiionships because I know the order by which the master genbank file was compiled.  

In [145]:
from Bio import SeqIO
from Bio import GenBank
import sqlite3

conn = sqlite3.connect("New.db")
c = conn.cursor()

geneNames = enzymes*3

geneValues = []
with open("MasterGenBankUPDATE.gb") as handle:
    count = 0
    for record in GenBank.parse(handle):
        value1 = (geneNames[count],)
        value2 = (record.definition,)
        value3 = (record.organism,)
        value4 = (record.sequence,)
        geneValues.append(value1+value2+value3+value4)
        count = count + 1    

for gene in geneValues:
    idNum = 1
    term = str(idNum) + ","+gene[0] + "," + gene[1]+","+gene[2]+","+gene[3]
    c.execute("""INSERT INTO genes (id, name, description, organism, nucleotide_sequence) VALUES("""+ term +");")
    idNum += 1
conn.commit()
print(c.fetchall())


for enzyme in enzymes:
    idNum=1
    term = str(idNum) + ","+enzyme+ ","+ EC_number[enzyme] 
    c.execute("""INSERT INTO enzymes (id, name, EC number) VALUES("""+ term +");")
    idNum+=1
    

pathways = ['Glycolysis', 'TCA cycle', 'Pentose Phosphate']
PWDescription = {'Glycolysis':'Glycolysis is the process of converting glucose into pyruvate and generating small amounts of ATP (energy) and NADH (reducing power).', 
                 'TCA cycle':'The citrate cycle (TCA cycle, Krebs cycle) is an important aerobic pathway for the final steps of the oxidation of carbohydrates and fatty acids.', 
                 'Pentose Phosphate': 'The pentose phosphate pathway is a process of glucose turnover that produces NADPH as reducing equivalents and pentoses as essential parts of nucleotides.'}
    
for pathway in pathways:
    idNum=1
    term = str(idNum) + ","+pathway+ ","+ PWDescription[pathway] 
    c.execute("""INSERT INTO pathways (id, name, description) VALUES("""+ term +");")
    idNum+=1


    

OperationalError: near "dehydrogenase": syntax error

I did not have time to move this lab any further, mostly due to syntax errors with SQL and database locking.  

'1'